In [1]:
import collections
import networkx as nx
import math

In [2]:
trial = open('trial.txt', 'r')
lines = [line for line in trial]
trial.close()

In [3]:
correctLines = []
for line in lines:
    if (((not line[0] == '\t') or ('...' in line)) and (line)):
        continue
    else:
        correctLines.append(line[4:line.find('(')])

В итоге, в correctLines находятся названия ошибок, которые мы будем трогать.

In [4]:
collisions = collections.Counter()

for line in correctLines:
    collisions[line] += 1
collisions

Counter({'o.e.j.s.HttpInput$ErrorState.noContent': 1,
         'o.e.j.s.HttpInput.read': 1,
         'o.g.j.m.i.EntityInputStream.read': 1,
         'o.g.j.m.i.ReaderInterceptorExecutor$UnCloseableInputStream.read': 1,
         'o.j.m.MIMEParser.fillBuf': 1,
         'o.j.m.MIMEParser.readBody': 1,
         'o.j.m.MIMEParser.access$600': 1,
         'o.j.m.MIMEParser$MIMEEventIterator.next': 2,
         'o.j.m.MIMEMessage.makeProgress': 1,
         'o.j.m.MIMEMessage.parseAll': 1,
         'o.j.m.MIMEMessage.getAttachments': 1,
         'o.g.j.m.m.i.MultiPartReaderClientSide.readMultiPart': 1,
         'o.g.j.m.m.i.MultiPartReaderServerSide.readMultiPart': 1,
         'o.g.j.m.m.i.MultiPartReaderClientSide.readFrom': 2,
         'o.g.j.m.i.ReaderInterceptorExecutor$TerminalReaderInterceptor.invokeReadFrom': 1,
         'o.g.j.m.i.ReaderInterceptorExecutor$TerminalReaderInterceptor.aroundReadFrom': 1,
         'o.g.j.m.i.ReaderInterceptorExecutor.proceed': 2,
         'o.g.j.s.i.Mappabl

collisions - по сути, словарик, в котором хранится, сколько раз мы столкнулись с той или иной ошибкой.

In [5]:
G = nx.Graph()

item_num = 0
uniqueCollisions = list(collisions)

while len(uniqueCollisions) > item_num: 
    current = uniqueCollisions[item_num]
    item_num += 1
    dotPlace = current.rfind('.')
    if dotPlace == -1:
        continue
    else:
        G.add_edge(current, current[:dotPlace])
        
        collisions[current[:dotPlace]] += collisions[current]
        uniqueCollisions = list(collisions)

Заполняем граф всеми рёбрами родитель-ребёнок (при добавлении edge, также добавляется вершина, если она ранее отсутствовала), также считаем, сколько раз встретился какой-то кусок, как сумму того, сколько раз суммарно встретились его дети первого порядка. Дозаполняем этими значениями наш словарь.

In [6]:
def children(graph, node, generation, childrenList):
    nodeLen = len(node)
    if generation == 0:
        childrenList.append(str(node))
        return childrenList
    else:
        for c_node, adjacencies in graph.adjacency():
            if not c_node == node:
                continue
            else:
                for candidate in adjacencies.keys():
                    if len(candidate) > nodeLen:
                        children(graph, candidate, generation-1, childrenList)
                break
    return childrenList

In [7]:
children(G, 'o', 8, [])

[]

In [8]:
stepRadius = 5
rootsNsizes = {}
for node in list(collisions):
    if node.find('.') == -1:
        currGeneration = 0
        while children(G, node, currGeneration, []):
#             print(str(node), end = "\t")
#             print(children(G, node, currGeneration, []), end = "\t")
#             print(currGeneration)
            currGeneration += 1
        rootsNsizes[node] = currGeneration - 1

rootsNsizes

{'o': 7}

In [9]:
def initiateRoots(graph, dictionary, radius):
    distance = -dictionary[list(dictionary)[0]]
    for root in dictionary.keys():
        distance += dictionary[root]
        graph.graph[root] = [distance * radius, 0]
        distance += (dictionary[root] + radius)

In [10]:
initiateRoots(G, rootsNsizes, stepRadius)
G.graph

{'o': [0, 0]}

Что имеется на данный момент? 

collisions - для каждого участка пути - сколько раз встретилось в логах.

Функция children - получение списка детей порядка n.

G.graph содержит координаты корней, полученные с помощью функции initiateRoots.

G.edges содержит значения рёбер, соединяющих имеющиеся вершины (в G оные ещё не особо записаны).

Что нужно сделать? Ввести все координаты в G.graph.

In [11]:
for r in list(G.graph):
    currentGeneration = 1
    currentRadius = stepRadius
    iterations = rootsNsizes[r]
    for i in range(iterations):
        nodesGen = children(G, r, currentGeneration, [])
        lenNodesGen = len(nodesGen)
        smallAngle = 2 * math.pi / lenNodesGen
        for nodeNum in range(lenNodesGen):
            G.graph[nodesGen[nodeNum]] = [G.graph[r][0] + math.cos(nodeNum * smallAngle) * currentRadius, G.graph[r][1] + math.sin(nodeNum * smallAngle)* currentRadius]
        currentRadius += stepRadius
        currentGeneration += 1

In [12]:
import matplotlib.pyplot as plt

x_values = []
y_values = []
for node in G.graph.keys():
    x_values.append(G.graph[node][0])
    y_values.append(G.graph[node][1])
    
plt.scatter(x_values, y_values, color = 'r')
plt.savefig("graphCheck", dpi = 1000)
plt.show()

<Figure size 640x480 with 1 Axes>

In [13]:
import plotly.plotly as py
import plotly.graph_objs as go

In [14]:
edge_trace = go.Scatter( #способ изображать линию
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'), #ширина и цвет линий
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.graph[edge[0]] # координаты точек первой вершины
    x1, y1 = G.graph[edge[1]] # координаты точек второй вершины
    edge_trace['x'] += tuple([x0, x1, None]) # добавление информации по координатам отрезка в проекции на x
    edge_trace['y'] += tuple([y0, y1, None]) # добавление информации по координатам отрезка в проекции на y

In [15]:
node_trace = go.Scatter(
    x=[], # положение точек по X
    y=[], # положение точек по Y
    text=[], # текст, появляющийся при наведении мышки на вершину
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10, # размер точек
        colorbar=dict( # описание характеристик цветовой полоски справа
            thickness=15, # ширина 
            title='none', #подпись
            xanchor='left', # изменяет отступ до полосы - "right" делает его меньше
            titleside='right' # положение текста к полосе... Оптимально - справа
        ),
        line=dict(width=2))) # ширина линии вокруг вершины

for node in G.nodes():
    x, y = G.graph[node]
    node_trace['x'] += tuple([x]) # добавляем проекции точек на ось координат x
    node_trace['y'] += tuple([y]) # добавляем проекции точек на ось координат y

In [16]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([collisions[list(adjacencies)[0]]])
    node_info = str(list(adjacencies)[0]) + ': '+ str(collisions[adjacencies[0]]) # название ошибки: 
    node_trace['text']+=tuple([node_info]) # выводим текст при наведении на 

In [17]:
import plotly.offline
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # чтообы оффлайн тоже всё работало

In [18]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Errors',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

plotly.offline.iplot(fig, filename='networkx')